In [1]:
%%capture
%pip install transformers trl

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="openai-community/gpt2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [3]:
outs = pipe('My name is Ellie and', num_return_sequences = 5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [4]:
type(outs)

list

In [5]:
from datasets import load_dataset

ds = load_dataset("econnors/conflictual")

conflictual_threads.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1606 [00:00<?, ? examples/s]

In [6]:
type(ds)

datasets.dataset_dict.DatasetDict

In [7]:
ds["train"]["comment"][0]

"522775351 I'm mostly pro male now Women have no race Unless you literally put women in shackles a good portion of men WILL be raising another man's child which defeats the entire purpose At least I'm raising my friends child he's White too No you're just a cuck and your bloodline is now ruined"

In [8]:
ds["train"]['comment'][:20]

["522775351 I'm mostly pro male now Women have no race Unless you literally put women in shackles a good portion of men WILL be raising another man's child which defeats the entire purpose At least I'm raising my friends child he's White too No you're just a cuck and your bloodline is now ruined",
 "522775704 You're on the wrong board if you think you'll be convince anyone that women are loyal France had to BAN PATERNITY TESTS because if men found out that up to half of them were raising another man's child men would revolt That's how bad it is Unless you literally put women in chains its impossible to claim your all White community is actually White because you're not DNA testing the children",
 '522773939 they are against race mixing kill yourself nigger lover',
 "522776290 522777213 This is why White Nationalism is laughable if women have any rights People point to Nazi Germany citizenship requiring a trace of your family blood line but did they DNA test everyone's children No so th

In [9]:
pipe(ds["train"]['comment'][10])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '522794170 2 Peter 3 3 9 Knowing this first that there shall come in the last days scoffers walking after their own lusts 4And saying Where is the promise of his coming for since the fathers fell asleep all things continue as they were from the beginning of the creation 5For this they willingly are ignorant of that by the word of God the heavens were of old and the earth standing out of the water and in the water 6Whereby the world that then was being overflowed with water perished 7But the heavens and the earth which are now by the same word are kept in store reserved unto fire against the day of judgment and perdition of ungodly men 8But beloved be not ignorant of this one thing that one day is with the Lord as a thousand years and a thousand years as one day 9The Lord is not slack concerning his promise as some men count slackness but is longsuffering to us ward not willing that any should perish but that all should come to repentance 10And the Lord says, "Now my

In [10]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)
from trl import SFTTrainer, SFTConfig

In [11]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m")


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [12]:
dataset = load_dataset("econnors/conflictual")

In [13]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [14]:
training_params = SFTConfig(
    output_dir="./results",
    num_train_epochs = 3, # how many times we iterate over the dataset as a whole
    learning_rate = 2e-4, # how many "steps" we take in adjusting the parameters to make up for loss
    weight_decay = 0.001, # way of regularizing the parameters
    dataset_text_field = "comment",
    report_to="none" # this is a new param, to avoid a login to W&B
)

In [15]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset['train'],
    processing_class = tokenizer,
    args = training_params
)

Adding EOS to train dataset:   0%|          | 0/1606 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1606 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1606 [00:00<?, ? examples/s]

In [16]:
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.


Step,Training Loss
10,4.968200
20,4.816500
30,4.980600
40,4.998300
50,4.790300
60,4.806000
70,4.821900
80,4.922400
90,4.651200
100,5.009200


TrainOutput(global_step=603, training_loss=3.906157395535243, metrics={'train_runtime': 786.8208, 'train_samples_per_second': 6.123, 'train_steps_per_second': 0.766, 'total_flos': 832908826552320.0, 'train_loss': 3.906157395535243, 'entropy': 3.7569872538248696, 'num_tokens': 707874.0, 'mean_token_accuracy': 0.35001473625500995, 'epoch': 3.0})

In [17]:
pipe("I'm so excited")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I\'m so excited for you!"\n\n"I really feel like I\'m in a big fight!"\n\n"Oh my god!"\n\n"I\'m going to be in such a big fight!"\n\nI was so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\n'}]

In [18]:
trainer.model.save_pretrained("models")
trainer.tokenizer.save_pretrained("models")

model = AutoModelForCausalLM.from_pretrained("./models")
tokenizer = AutoTokenizer.from_pretrained("./models")

pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=50)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Device set to use cuda:0


In [19]:
trainer.save_model("./conflictual_model")
tokenizer.save_pretrained("./conflictual_model")

('./conflictual_model/tokenizer_config.json',
 './conflictual_model/special_tokens_map.json',
 './conflictual_model/vocab.json',
 './conflictual_model/merges.txt',
 './conflictual_model/added_tokens.json',
 './conflictual_model/tokenizer.json')

In [20]:
pipe("Gender")

[{'generated_text': 'Gender books are a hyperborea s book that you re reading for yourself and your spouse It s also a hyperborea s book that you re reading for your spouse It s also a hyperborea s book that you re reading for your spouse'}]

In [21]:
pipe("Incel")

[{'generated_text': "Incel material is a very literal expression of human sexuality and sex and should be banned from being used in certain contexts such as pornography or by consenting adults especially if it's used in a medical condition such as heart disease or certain sexually explicit terms like"}]

In [23]:
!zip -r conflictual_model.zip conflictual_model
from google.colab import files
files.download("conflictual_model.zip")

  adding: conflictual_model/ (stored 0%)
  adding: conflictual_model/training_args.bin (deflated 53%)
  adding: conflictual_model/model.safetensors (deflated 8%)
  adding: conflictual_model/generation_config.json (deflated 31%)
  adding: conflictual_model/vocab.json (deflated 59%)
  adding: conflictual_model/special_tokens_map.json (deflated 80%)
  adding: conflictual_model/tokenizer.json (deflated 82%)
  adding: conflictual_model/tokenizer_config.json (deflated 55%)
  adding: conflictual_model/config.json (deflated 59%)
  adding: conflictual_model/merges.txt (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>